In [ ]:
import asyncio
import client as async_client # the asynchronous inner workings is in client.py
import pandas as pd
import sqlite3
import datetime
from dateutil import tz
import os

In [ ]:
# get the battle logs this is the first time uses the leadboard as a seed
database = "databases/data"
client = async_client.Client(database)
ids = await client.get_ids("medium")
client.add_battle_logs_to_database(ids)

In [ ]:
# recursively get battle logs using the ids from the battle logs from the first piece of code
database = "databases/data"
client = async_client.Client(database)
df = pd.read_sql("SELECT team1_player1_tag, team1_player2_tag, team1_player3_tag, team2_player1_tag, team2_player2_tag, team2_player3_tag from battle_logs", con = sqlite3.connect("databases/old_data"))
ids = df.values.flatten()
add_battle_logs_to_database(self, ids)

In [ ]:
# becaouse there are too many games and it would take too long to get all information for all the player ids, filter for games that were above 600 trophies and store them into a cleaned database
df = pd.read_sql("SELECT * from battle_logs", sqlite3.connect("databases/data"))
df["hash"] = df["time"].apply(hash) + df["mode"].apply(hash) + df["map"].apply(hash)
df = df.drop_duplicates("hash")
df["time"] = pd.to_datetime(df["time"], utc = True)
df["time"].dt.tz_convert(tz.tzlocal()).head()
df = df[(df[["team1_player1_trophies", "team1_player2_trophies", "team1_player3_trophies"]].max(axis = 1) > 600) & (df[["team2_player1_trophies", "team2_player2_trophies", "team2_player3_trophies"]].max(axis = 1) > 600)]
df.to_sql("battle_logs", con = sqlite3.connect("databases/data2"), if_exists = "append", index = False)

In [ ]:
# run this block if it is the first time getting player information
database = "databases/data"
client = async_client.Client(database)
df = pd.read_sql("SELECT team1_player1_tag, team1_player2_tag, team1_player3_tag, team2_player1_tag, team2_player2_tag, team2_player3_tag from battle_logs", con = sqlite3.connect("databases/old_data"))
await client.add_player_data_to_databases(df.values.flatten())

In [ ]:
# run this if the cell above timed out; adds the ids that have not been accessed yet
database = "databases/data"
client = async_client.Client(database)
df = pd.read_sql("SELECT team1_player1_tag, team1_player2_tag, team1_player3_tag, team2_player1_tag, team2_player2_tag, team2_player3_tag from battle_logs", con = sqlite3.connect("databases/old_data"))
df2 = pd.read_sql("SELECT * from players", con = sqlite3.connect("databases/data"))
ids = set(df.values.flatten())
already_finished_ids = df2["tag"].values.flatten()
already_finished_ids = set(already_finished_ids)
ids = ids.difference(already_finished_ids)
await client.add_player_data_to_databases(ids)